In [5]:
%pip install pyspark==3.5.3 py4j==0.10.9.7
%pip install wget

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [6]:
from pyspark.sql import SparkSession
import os, sys, wget

In [3]:
# 1) 로컬 호스트 IP 구하기
host_ip = "127.0.0.1"  # 또는 socket.gethostbyname(socket.gethostname())

# 2) 환경변수 지정
os.environ['SPARK_LOCAL_IP']           = host_ip
os.environ['PYSPARK_PYTHON']           = os.sys.executable
os.environ['PYSPARK_DRIVER_PYTHON']    = os.sys.executable
os.environ["HADOOP_HOME"] = "C:\\Hadoop\\hadoop-3.3.6"


PySpark Py4JJavaError : write.csv 오류 해결

> python 3.11.9 버전 호환
https://www.reddit.com/r/apachespark/comments/1evu4zz/error_with_pyspark_and_py4j/

> hadoop.dll 파일 설치
https://www.reddit.com/r/apachespark/comments/18ftcgs/unable_to_write_dataframe_to_files_using_pyspark/

In [22]:
from pyspark import SparkConf

conf = SparkConf()
conf.set("spark.app.name", "PySpark DataFrame #3")
conf.set("spark.master", "local[*]")
conf.set("spark.driver.host", host_ip)
conf.set("spark.driver.bindAddress", host_ip)

spark = SparkSession.builder\
        .config(conf=conf)\
        .getOrCreate()

In [9]:
save_path = 'data/transfer_cost.txt'
download_path = 'https://s3-geospatial.s3-us-west-2.amazonaws.com/transfer_cost.txt'

if not os.path.exists(save_path):
    wget.download(download_path, out=save_path)

In [10]:
!chcp 65001
!dir data

Active code page: 65001
 Volume in drive C has no label.
 Volume Serial Number is BC10-1C56

 Directory of c:\Users\chaeyni\Documents\study\spark\programming-pyspark\chapter2\data

2025-07-28  오후 10:01    <DIR>          .
2025-07-27  오후 11:53    <DIR>          ..
2025-07-27  오후 05:13            62,728 1800.csv
2025-07-27  오후 10:27           146,855 customer-orders.csv
2025-07-28  오후 10:01    <DIR>          extracted.csv
2025-07-27  오후 05:04               997 name_gender.csv
2025-07-27  오후 11:37           286,779 transfer_cost.txt
               4 File(s)        497,359 bytes
               3 Dir(s)  614,474,407,936 bytes free


In [40]:
!powershell -Command "Get-Content data/transfer_cost.txt | Select-Object -First 5"

On 2021-01-04 the cost per ton from 85001 to 85002 is $28.32 at ABC Hauling
On 2021-01-04 the cost per ton from 85001 to 85004 is $25.68 at ABC Hauling
On 2021-01-04 the cost per ton from 85001 to 85007 is 19.86 at ABC Hauling
On 2021-01-04 the cost per ton from 85001 to 85007 is 20.52 at Haul Today
On 2021-01-04 the cost per ton from 85001 to 85010 is 20.72 at Haul Today


In [12]:
import pyspark.sql.functions as F
from pyspark.sql.types import *

transfer_cost_txt = save_path

schema = StructType([ StructField("text", StringType(), True)])
transfer_cost_df = spark.read.schema(schema).text(transfer_cost_txt)

In [13]:
transfer_cost_df.show(truncate=False)

+---------------------------------------------------------------------------+
|text                                                                       |
+---------------------------------------------------------------------------+
|On 2021-01-04 the cost per ton from 85001 to 85002 is $28.32 at ABC Hauling|
|On 2021-01-04 the cost per ton from 85001 to 85004 is $25.68 at ABC Hauling|
|On 2021-01-04 the cost per ton from 85001 to 85007 is 19.86 at ABC Hauling |
|On 2021-01-04 the cost per ton from 85001 to 85007 is 20.52 at Haul Today  |
|On 2021-01-04 the cost per ton from 85001 to 85010 is 20.72 at Haul Today  |
|On 2021-01-04 the cost per ton from 85001 to 85012 is $18.98 at ABC Hauling|
|On 2021-01-04 the cost per ton from 85001 to 85013 is 26.64 at Haul Today  |
|On 2021-01-04 the cost per ton from 85001 to 85020 is 26.34 at ABC Hauling |
|On 2021-01-04 the cost per ton from 85001 to 85021 is $20.15 at ABC Hauling|
|On 2021-01-04 the cost per ton from 85002 to 85001 is 21.57 at 

In [14]:
from pyspark.sql.functions import *
regex_str = r'On (\S+) the cost per ton from (\d+) to (\d+) is (\S+) at (.*)'

df_with_new_columns = transfer_cost_df\
                        .withColumn('week', regexp_extract('text', regex_str, 1))\
                        .withColumn('departure_zipcode', regexp_extract(column('text'), regex_str, 2))\
                        .withColumn('arrival_zipcode', regexp_extract(transfer_cost_df.text, regex_str, 3))\
                        .withColumn('cost', regexp_extract(col('text'), regex_str, 4))\
                        .withColumn('vendor', regexp_extract(col('text'), regex_str, 5))

In [15]:
df_with_new_columns.printSchema()

root
 |-- text: string (nullable = true)
 |-- week: string (nullable = true)
 |-- departure_zipcode: string (nullable = true)
 |-- arrival_zipcode: string (nullable = true)
 |-- cost: string (nullable = true)
 |-- vendor: string (nullable = true)



In [16]:
final_df = df_with_new_columns.drop("text")

In [17]:
final_df

DataFrame[week: string, departure_zipcode: string, arrival_zipcode: string, cost: string, vendor: string]

In [ ]:
save_path = "data/extracted.csv"

if not os.path.exists(save_path):
    final_df.write.csv(save_path)

In [24]:
!dir data

 Volume in drive C has no label.
 Volume Serial Number is BC10-1C56

 Directory of c:\Users\chaeyni\Documents\study\spark\programming-pyspark\chapter2\data

2025-08-02  오후 03:07    <DIR>          .
2025-07-27  오후 11:53    <DIR>          ..
2025-07-27  오후 05:13            62,728 1800.csv
2025-07-27  오후 10:27           146,855 customer-orders.csv
2025-08-02  오후 03:07    <DIR>          extracted.csv
2025-07-27  오후 05:04               997 name_gender.csv
2025-07-27  오후 11:37           286,779 transfer_cost.txt
               4 File(s)        497,359 bytes
               3 Dir(s)  613,761,634,304 bytes free


In [33]:
!dir data\extracted.csv

 Volume in drive C has no label.
 Volume Serial Number is BC10-1C56

 Directory of c:\Users\chaeyni\Documents\study\spark\programming-pyspark\chapter2\data\extracted.csv

2025-08-02  오후 03:07    <DIR>          .
2025-08-02  오후 03:07    <DIR>          ..
2025-08-02  오후 03:07             1,264 .part-00000-bdff1522-8c10-4aeb-b480-e546020362f6-c000.csv.crc
2025-08-02  오후 03:07                 8 ._SUCCESS.crc
2025-08-02  오후 03:07           160,257 part-00000-bdff1522-8c10-4aeb-b480-e546020362f6-c000.csv
2025-08-02  오후 03:07                 0 _SUCCESS
               4 File(s)        161,529 bytes
               2 Dir(s)  613,759,090,688 bytes free


In [45]:
!powershell -Command "Get-Content data/extracted.csv/part-00000-bdff1522-8c10-4aeb-b480-e546020362f6-c000.csv | Select-Object -First 5"

2021-01-04,85001,85002,$28.32,ABC Hauling
2021-01-04,85001,85004,$25.68,ABC Hauling
2021-01-04,85001,85007,19.86,ABC Hauling
2021-01-04,85001,85007,20.52,Haul Today
2021-01-04,85001,85010,20.72,Haul Today


In [46]:
save_path_json = "data/extracted.json"

if not os.path.exists(save_path_json):
    final_df.write.format("json").save(save_path_json)

In [47]:
!dir data\extracted.json

 Volume in drive C has no label.
 Volume Serial Number is BC10-1C56

 Directory of c:\Users\chaeyni\Documents\study\spark\programming-pyspark\chapter2\data\extracted.json

2025-08-02  오후 03:29    <DIR>          .
2025-08-02  오후 03:29    <DIR>          ..
2025-08-02  오후 03:29             3,420 .part-00000-aa8cfc40-7d09-4b3d-82c7-fe08074e94b4-c000.json.crc
2025-08-02  오후 03:29                 8 ._SUCCESS.crc
2025-08-02  오후 03:29           436,305 part-00000-aa8cfc40-7d09-4b3d-82c7-fe08074e94b4-c000.json
2025-08-02  오후 03:29                 0 _SUCCESS
               4 File(s)        439,733 bytes
               2 Dir(s)  613,751,742,464 bytes free


In [48]:
!powershell -Command "Get-Content data/extracted.json/part-00000-aa8cfc40-7d09-4b3d-82c7-fe08074e94b4-c000.json | Select-Object -First 1"

{"week":"2021-01-04","departure_zipcode":"85001","arrival_zipcode":"85002","cost":"$28.32","vendor":"ABC Hauling"}
